<a href="https://colab.research.google.com/github/Base-R-Best-R/Auction/blob/main/Code/Models/Colab/RF_Train_Sk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RF in Sklearn

In [5]:
# Imports
import pandas as pd
import numpy as np
# !pip install pyreadr
import pyreadr
import sklearn

In [ ]:
# Import RDS training Data